In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab06.ipynb")

# Lab 6: Message Authentication Codes and Digital Signatures
Contributions From: Ryan Cottone

Welcome to Lab 6! In this lab, you will build a HMAC function and RSA signature, and demonstrate the dangers of textbook RSA signatures!

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install pycrypto

In [ ]:
from Crypto.Hash import SHA256

# Message Authentication Codes

Message Authentication Codes (hereafter MACs) function as a 'tag' that allows us to ensure the message was not modified in transit. Those who wish to communicate will share a secret key $K$. To send a message, Alice picks $M$ and computes $\text{MAC}(K, M)$, finally sending over $(M, \text{MAC}(K, M))$. Note that this does not hide the message! MACs only provide integrity against tampering.

If Mallory tried to change $M$ to $M'$ mid message, she would also need to compute $\text{MAC}(K, M')$. Fortunately, this is considered to be very difficult for a secure MAC. Were she to modify the message to be $(M', \text{MAC}(K, M)$), Bob would compute $\text{MAC}(K, M')$ and see that it does not match up with $\text{MAC}(K, M)$.

## HMAC

The HMAC is a class of MAC that utilizes a cryptographically-secure hash function to create integrity. The HMAC function is as follows: $$\text{HMAC}(K, M) = H((K \oplus \text{opad}) || H((K \oplus \text{ipad}) || M))$$.

The opad and ipad are just constants there to make K into two different keys. Read Note 6 for more information!

**Question 1**: Implement the HMAC!

In [ ]:
def bxor(b1, b2): # use xor for bytes
    result = bytearray()
    for b1, b2 in zip(b1, b2):
        result.append(b1 ^ b2)
    return result

def HMAC(K, M):
    assert len(K) == 32
    
    h = SHA256.new()
    
    # Compute K xor ipad 
    K_inner = bxor(K, b'V'*32)
    
    # Compute K xor opad 
    K_outer = bxor(K, b'|'*32)
    
    M = bytes(M, 'UTF-8')
              
    # Compute H(K_inner || M)
    # Remember, we can add two byte string like a + b!
    hash_argument = ...
    
    h.update(hash_argument)
    
    # Compute H(K_outer || inner_hash)
    inner_hash = h.digest()
    hash_argument = ...
    
    outer_hash = h.update(hash_argument)
    
    return h.digest()

In [ ]:
grader.check("q1_1")

**Question 1.2**: Implement a MAC verifier.

In [ ]:
def verifyMessage(K, M, HMAC_M):
    ...
        return True
    else:
        return False

In [ ]:
grader.check("q1_2")

You may be wondering why we have such a complicated setup and not just use $\text{MAC}(K, M) = H(K || M)$. Unfortunately, this is susceptible to the exact same length-extension attack you did in the last lab! An atacker would be able whatever data they wanted to the end of the message.

# Digital Signatures 

Another setup which provides both **integrity** and **authenticity** is the digital signature, a way of "signing" a message to verify it came from a specific person. Broadly, Alice will publish a public key and keep a secret key. She signs a message with her secret key, and everyone else can verify that the message is signed by the secret key corresponding to her public key. Formally: $S = \text{Sign}(S_k, M)$ creates a signature on message $M$ signed by secret key $S_k$, and $\text{Verify}(S, M, P_k)$ returns True if the message was signed by the owner of $P_k$. 

Crucially, Mallory cannot modify $M$ and retain a valid signature or try to pass off some other message $M'$ as signed by Alice.

## RSA Signatures

One of the most basic signature schemes is that of textbook RSA signatures. In this scheme, Alice has an RSA keypair of $e$ as her public key and $d$ as her private key (exactly the same as RSA). She signs a message like:

$$\text{Sign}(d, M, N) = M^d \mod N$$

and one can verify the message is legitimate via:

$$\text{Verify}(e, S, M, N)= (S^e \stackrel{?}{=} M \mod N)$$

Notice that $S = M^d$, so $S^e \equiv (M^d)^e \equiv M^{ed} \equiv M \mod N$, as specified in RSA.

Note that this is insecure for reasons we will soon see!

**Question 2.1**: Implement the Textbook RSA Signature Scheme!

In [ ]:
# Finds M^d mod N. Use pow(base, exponent, modulus)
def sign(d, M, N):
    ...

# Finds S^e mod N and compares it to M. Use pow(base, exponent, modulus)
def verify(e, S, N, M):
    ...

In [ ]:
grader.check("q2_1")

Now that we've setup our (faulty) signature scheme, let's demonstrate a forgery attack on it! From the notes/lecture, you may know that we can set some $M \in [0, N-1]$ to be our signature, and our message will therefore be $M^e$.

$$S = M^d \implies M = S^e$$

Unfortunately, we can't choose the message to be whatever we want, but we can bruteforce it to some extent!

In [ ]:
def getExpansion(n,m):
    arr = []
    
    while n > 0:
        r = n % m
        n //= m
        
        arr.append(r)
    
    return arr

def textToInt(s):
    total = 0
    
    for i in range(len(s)):
        total += ord(s[i])*(256**i)
    
    return total

def intToText(n):
    expansion = getExpansion(n, 256)
    
    finalStr = ""
    
    for i in range(len(expansion)):
        finalStr += chr(expansion[i])
        
    return finalStr

In [ ]:
N = 57827237212537328721403414859701117256950147043657179682429675635414974092267
p = 197392385385765730323593372879610033749
q = 292955764729855380124638330592302833983
e = 30976423168144014209716624790587982344466107184812898368404437795227984190221
d = 25506717026652416005253828955620679434349390414876094959093739255385916016069


# Pretend we are Mallory and don't know Alice's private key, but still want to forge a signature.
mock_signature = 111122223333344444

message = pow(mock_signature, e, N)
print('Derived message:', intToText(message))

print('\nChecking message...\n')
if verify(e, mock_signature, N, message):
    print('Message verified as being from Alice!')
else:
    print('Message failed signature verification!')

Of course, this is just gibberish, but since we were able to make the program think it was from Alice, we've broken the scheme entirely.

The correct way to build sign is: $$\text{Sign}(d, M) = H(M)^d \mod N$$
where $H$ is some cryptographic hash function. Verification is as following:

$$\text{Verify}(e, S, M, N)= (S^e \stackrel{?}{=} H(M) \mod N)$$

In [ ]:
## This code is given to you!

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def H(M):
    h = SHA256.new()
    h.update(bytes(int_to_bytes(M)))
    return int(h.hexdigest(), 16)

# Finds H(M)^d mod N.
def secureSign(d, M, N):
    return pow(H(M), d, N)

# Finds S^e mod N and compares it to H(M).
def secureVerify(e, S, N, M):
    return pow(S, e, N) == H(M)

If we try our attack from earlier, we see that it no longer works:

In [ ]:
mock_signature = 111122223333344444

message = pow(mock_signature, e, N)
print('Derived message:', intToText(message))

print('\nChecking message...\n')
if secureVerify(e, mock_signature, N, message):
    print('Message verified as being from Alice!')
else:
    print('Message failed signature verification!')

To make this work, we'd need to find some input $x$ to $H$ such that $H(x) = M$, which is hard under collision-resistant functions.

Congrats on finishing Lab 6!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have generated the zip file, go to the Gradescope page for this assignment to submit.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)